In [3]:
import webbrowser, sys
from bs4 import BeautifulSoup
import requests
import csv

In [5]:
pageScrape = requests.get("https://montanafreepress.org/")
soup = BeautifulSoup(pageScrape.text, "html.parser")
print(soup)

<!DOCTYPE html>

<html lang="en-US">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<link href="https://gmpg.org/xfn/11" rel="profile"/>
<meta content="index, follow, max-image-preview:large, max-snippet:-1, max-video-preview:-1" name="robots"/>
<style>img:is([sizes="auto" i],[sizes^="auto," i]){contain-intrinsic-size: 3000px 1500px}</style>
<script async="" data-cfasync="false" data-no-defer="1" data-no-minify="1" data-no-optimize="1" src="https://securepubads.g.doubleclick.net/tag/js/gpt.js" type="pmdelayedscript"></script>
<script>
			window.googletag = window.googletag || { cmd: [] };
		</script>
<link href="https://i0.wp.com" rel="preconnect"/>
<script>window._wca = window._wca || [];</script>
<title>Montana's Independent Source for Statewide News - Montana Free Press</title><link as="image" fetchpriority="high" href="https://montanafreepress.org/wp-content/uploads/2020/05/mtfp-logo-1.png" imagesizes="(max-width: 860px) 100vw, 